<a href="https://colab.research.google.com/github/dayoxy/git-daxy/blob/main/MNIST_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow numpy matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, _), (_, _) = mnist.load_data()

# Normalize the images to [-1, 1] range
x_train = (x_train.astype('float32') - 127.5) / 127.5  # Normalize to [-1, 1]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)  # Reshape to (28, 28, 1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras import layers, models

def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(7 * 7 * 256, input_dim=100))  # Input: 100-dimensional noise vector
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.ReLU())
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.ReLU())
    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.ReLU())
    model.add(layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='tanh'))  # Output: (28, 28, 1)

    return model


In [ ]:
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))  # Output: probability of being real or fake

    return model


In [ ]:
from tensorflow.keras.optimizers import Adam

# Loss function (Binary Cross-Entropy)
cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Optimizer (Adam)
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)


In [ ]:
import numpy as np

def train_gan(generator, discriminator, epochs=10000, batch_size=64):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        real_images = x_train[idx]
        fake_images = generator.predict(np.random.randn(half_batch, 100))

        # Labels
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Discriminator training
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator (via Discriminator's feedback)
        noise = np.random.randn(batch_size, 100)
        valid_labels = np.ones((batch_size, 1))  # Want the Generator to fool the Discriminator
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % 1000 == 0:
            print(f"Epoch {epoch} | D Loss: {d_loss} | G Loss: {g_loss}")




In [ ]:
import matplotlib.pyplot as plt

def plot_generated_images(generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.randn(examples, 100)
    generated_images = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], cmap='gray')
        plt.axis('off')
    plt.show()


In [ ]:
plt.imsave('/content/generated_image.png', generated_images[0, :, :, 0], cmap='gray')


NameError: name 'generated_images' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ... (Your existing code: build_generator, build_discriminator, train_gan, plot_generated_images) ...

# Build and train the GAN
generator = build_generator()
discriminator = build_discriminator()

# Assume 'gan' is defined as a combined GAN model
# For example: gan = ... (Your GAN model definition) ...
# If you haven't defined 'gan' yet, you'll need to do so.
# Usually this is done by combining the generator and discriminator in a specific way

train_gan(generator, discriminator) # train the GAN

# Generate images and plot them
plot_generated_images(generator)  # This line now defines 'generated_images'

# Save the first generated image
plt.imsave('/content/generated_image.png', generated_images[0, :, :, 0], cmap='gray')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step


ValueError: You must call `compile()` before using the model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ... (Your existing code: build_generator, build_discriminator, train_gan, plot_generated_images) ...

# Build and train the GAN
generator = build_generator()
discriminator = build_discriminator()

# Compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy']) # Add this line

# Assume 'gan' is defined as a combined GAN model
# For example: gan = ... (Your GAN model definition) ...
# If you haven't defined 'gan' yet, you'll need to do so.
# Usually this is done by combining the generator and discriminator in a specific way

train_gan(generator, discriminator) # train the GAN

# Generate images and plot them
plot_generated_images(generator)  # This line now defines 'generated_images'

# Save the first generated image
plt.imsave('/content/generated_image.png', generated_images[0, :, :, 0], cmap='gray')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step


InvalidArgumentError: Graph execution error:

Detected at node sequential_3_1/dense_3_1/MatMul defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/di